In [90]:
import os
import numpy as np
import ast
from sklearn.metrics import roc_curve, roc_auc_score

In [91]:
def get_auc_from_file(fpath):
    with open(fpath, "r") as f:
        f.readline()
        auc = f.readline()
    return float(auc.strip().split("=")[-1])

def get_ytrueNpreds_from_file(fpath):
    with open(fpath, "r") as f:
        lines = f.readlines()
        y_true, y_pred = lines[10], lines[12]

    return ast.literal_eval(y_true.split("=")[-1]), ast.literal_eval(y_pred.split("=")[-1])

In [92]:
import pandas as pd
import numpy as np
import scipy.stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return z, np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    print(auc[0])
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

In [93]:
fpath2results = "./Results/"
txt_files_in_results = [f for f in os.listdir(fpath2results) if ".txt" in f]

avg_files_Meso = [f"{fpath2results}{f}" for f in txt_files_in_results if "v2avg.txt" in f and "Meso4" in f]
rnd_files_Meso = [f"{fpath2results}{f}" for f in txt_files_in_results if "v2rnd.txt" in f and "Meso4" in f]
avg_files_Inception = [f"{fpath2results}{f}" for f in txt_files_in_results if "v2avg.txt" in f and "Inception" in f]
rnd_files_Inception = [f"{fpath2results}{f}" for f in txt_files_in_results if "v2rnd.txt" in f and "Inception" in f]
run_avg_files_Meso = [f"{fpath2results}{f}" for f in txt_files_in_results if "runningavg.txt" in f and "Meso4" in f]
run_rnd_files_Meso = [f"{fpath2results}{f}" for f in txt_files_in_results if "runningrnd.txt" in f and "Meso4" in f]
run_avg_files_Inception = [f"{fpath2results}{f}" for f in txt_files_in_results if "runningavg.txt" in f and "Inception" in f]
run_rnd_files_Inception = [f"{fpath2results}{f}" for f in txt_files_in_results if "runningrnd.txt" in f and "Inception" in f]

In [94]:
# Only Meso4
avg_pairs = rnd_pairs = np.asarray([])
count = 0
for filea, fileb in zip(avg_files_Meso, rnd_files_Meso):
    avg_y_true, avg_y_pred = get_ytrueNpreds_from_file(filea)
    for a, b in zip(avg_y_true, avg_y_pred):
        avg_pairs=np.append(avg_pairs,((a,b)))
    rnd_y_true, rnd_y_pred = get_ytrueNpreds_from_file(fileb)
    for a, b in zip(rnd_y_true, rnd_y_pred):
        rnd_pairs=np.append(rnd_pairs,((a,b)))
avg_pairs = np.reshape(avg_pairs,(-1,2))
rnd_pairs = np.reshape(rnd_pairs,(-1,2))

In [95]:
all(rnd_pairs[:,0] == avg_pairs[:,0])

True

In [96]:
# Other_Model
avg_pairs = rnd_pairs = np.asarray([])
for filea, fileb in zip(avg_files_Inception, rnd_files_Inception):
    avg_y_true, avg_y_pred = get_ytrueNpreds_from_file(filea)
    for a, b in zip(avg_y_true, avg_y_pred):
        avg_pairs=np.append(avg_pairs,((a,b)))
    rnd_y_true, rnd_y_pred = get_ytrueNpreds_from_file(fileb)
    for a, b in zip(rnd_y_true, rnd_y_pred):
        rnd_pairs=np.append(rnd_pairs,((a,b)))
avg_pairs = np.reshape(avg_pairs,(-1,2))
rnd_pairs = np.reshape(rnd_pairs,(-1,2))

In [97]:
# rnd_pairs[:,0][:10], rnd_pairs[:,1][:10], avg_pairs[:,1][:10]
len_preds = 1000000000
# for t, p1, p2 in zip(rnd_pairs[:,0][:len_preds], rnd_pairs[:,1][:len_preds], avg_pairs[:,1][:len_preds]):
#     print(f"{int(t)}, {int(p1>.5)}, {int(p2>.5)}")

In [98]:
z_score_Inception, p_value_Inception = delong_roc_test(rnd_pairs[:,0][:len_preds], rnd_pairs[:,1][:len_preds], avg_pairs[:,1][:len_preds])
z_score_Meso, p_value_Meso = delong_roc_test(rnd_pairs[:,0][:len_preds], rnd_pairs[:,1][:len_preds], avg_pairs[:,1][:len_preds])


print(10**p_value_Inception, 10**p_value_Meso)
# print(z_score_Inception, z_score_Meso)

[[1.45983521e-07]] [[1.45983521e-07]]


In [99]:
z_score_Inception, p_value_Inception = delong_roc_test(rnd_pairs[:,0], rnd_pairs[:,1], avg_pairs[:,1])
z_score_Meso, p_value_Meso = delong_roc_test(rnd_pairs[:,0], rnd_pairs[:,1], avg_pairs[:,1])


print(10**p_value_Inception, 10**p_value_Meso)
# print(z_score_Inception, z_score_Meso)
# Under the null hypothesis, z can be well approximated by the standard normal distribution. 
# Therefore, if the value of z deviates too much from zero, e.g., z > 1.96, it is thus reasonable 
# to consider that [theta(A) > theta(B)] with the significance level p < 0.05.

# Here theta(A) is model rnd and theta(B) is model avg
print(10**p_value_Inception, 10**p_value_Meso)

[[1.45983521e-07]] [[1.45983521e-07]]
